In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from statistics import mean
from datetime import timedelta

import math

import warnings  
warnings.filterwarnings('ignore')

In [8]:
data = pd.read_csv("~/Desktop/NCSA_genomics/Python - notebooks/TSForecasting/Data/consolidatedDataForPackage.csv")
data['Display Time'] = pd.to_datetime(data['Display Time'])
data['GlucoseValue'] = pd.to_numeric(data['GlucoseValue'])

In [9]:
def fullDay(data):
        
    dates = list()
    data = data.reset_index(drop=True)
    for i in range(0,len(data.index)):
        dates.append(data['Display Time'][i].date())
    data['Dates'] = dates
    end = data['Dates'].iloc[-1]
    start = data['Dates'].iloc[0]

    indexVals = data[ data['Dates'] == start ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    indexVals = data[ data['Dates'] == end ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    data = data.reset_index(drop=True)

    data.drop(['Dates'], axis=1, inplace=True)

    return data

In [24]:
def IGC(df, unit, lltr = 80, ultr = 140, a = 1.1, b = 2.0, c = 30, d = 30):
    if unit == 'mg':
        gv = df['GlucoseValue']
    elif unit == 'mmol':
        gv = 18*df['GlucoseValue']
    else:
        print('Unit should either be mg or mmol')
        return 0
    
    lower_gv = gv[gv < 90]
    upper_gv = gv[gv > 140]
    
    
    count_lower = len(lower_gv.index)
    count_upper = len(upper_gv.index)
    
    hypoglycemicIndex = np.sum(np.power((lltr - lower_gv), b)) / (count_lower*d)   
    hyperglycemicIndex = np.sum(np.power((upper_gv - ultr), a)) / (count_upper*c)
    
    if np.isnan(hypoglycemicIndex):
        hypoglycemicIndex = 0
    if np.isnan(hyperglycemicIndex):
        hyperglycemicIndex=0
    
    igc = hypoglycemicIndex + hyperglycemicIndex
    return igc, hypoglycemicIndex, hyperglycemicIndex

In [25]:
for subjectId, df in data.groupby('subjectId'):
    df = fullDay(df)
    igc, hypoglycemicIndex, hyperglycemicIndex = IGC(df, 'mg')
    print(subjectId, igc, hypoglycemicIndex, hyperglycemicIndex)

1636-69-001 3.079991695297878 0.9774774774774775 2.1025142178204006
1636-69-001-2 1.9058920616911923 1.228985507246377 0.6769065544448154
1636-69-026 1.972940320981948 1.3972549019607843 0.5756854190211638
1636-69-028 1.6663131227858643 1.2833333333333334 0.3829797894525308
1636-69-032 1.740718384240811 1.4662309368191722 0.2744874474216389
1636-69-035 1.811577342963008 1.0314814814814814 0.7800958614815265
1636-69-048 1.20625 1.20625 0
1636-69-053 1.9709739489569493 1.1265745007680492 0.8443994481889001
1636-69-060 0.5833807945671072 0 0.5833807945671072
1636-69-064 2.1178485410635823 1.3251572327044026 0.7926913083591799
1636-69-069 1.9051583034271276 1.1981012658227848 0.7070570376043429
1636-69-090 1.921861709775849 0.8731578947368421 1.048703815039007
1636-69-091 1.8890993021540887 1.4457912457912458 0.4433080563628429
1636-69-100 1.5711704561433948 1.228840579710145 0.3423298764332499
1636-69-104 2.009371498391732 1.7502808988764045 0.2590905995153274
1636-69-104-2 4.403093434343